# Multivariante del método K-Nearest Neighbors

Hasta ahora exploramos como usar un modelo de ML, el KNN simple que uso solo una caracteristica para las predicciones. Si bien esto ayudo a familiarizarse con los conceptos basicos del ML, esta claro que usar una sola caracteristica no refleja la realidad.<br>

Hay 2 formas en las que podemos retocar este modelo para tratar de mejorar la precision (mejorar el RMSE):<br>
- Incrementar el numero de atributos (**caracteristicas**) que el modelo usa para calcular similitudes cuando ranque los vecinos mas cercanos.
- Incrementar **k**, el numero de vecinos cercanos que el modelo usa.

### Eliminacion de caracteristicas no utiles y _**Normalizacion**_ de los datos.

Al seleccionar mas atributos, tenemos que tener cuidado con las columnas que no funcionan bien con la ecuacion de distancia:
- Valores no numericos (ejemplo, ciudad o estado)
    - La ecuacion de distancia euclidiana espera valores numericos.
- Valores perdidos
    - La ecuacion de la distancia euclideana espera un valor por cada observacion y atributo.
- Valores no ordinales (por ejemplo, latitud o longitud)
    - La clasifiacion por distancia euclideana no tiene sentido si todos los valores no son ordinales. 

In [2]:
import pandas as pd
import numpy as np

# Leyendo datos y acomodandolos aleatoreamente
dc_listings = pd.read_csv('dc_airbnb.csv')
dc_listings = dc_listings.loc[ np.random.permutation(len(dc_listings)) ]

# Convirtiendo la columna price a float (limpiando datos)
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')

dc_listings.head()

,host_response_rate,host_acceptance_rate,host_listings_count,accommodates,room_type,bedrooms,bathrooms,beds,price,cleaning_fee,security_deposit,minimum_nights,maximum_nights,number_of_reviews,latitude,longitude,city,zipcode,state
3390,100%,100%,1,1,Private room,1.0,1.0,1.0,99.0,$30.00,NaN,2,1125,26,38.927722,-77.057276,Washington,20008,DC
3168,100%,90%,1,2,Private room,1.0,1.0,1.0,65.0,NaN,NaN,1,5,4,38.928664,-77.028240,Washington,20009,DC
3631,98%,52%,49,3,Entire home/apt,1.0,1.0,2.0,175.0,NaN,NaN,3,14,1,38.889065,-76.993576,Washington,20003,DC
187,58%,51%,480,2,Entire home/apt,1.0,1.0,1.0,239.0,$150.00,NaN,5,730,0,38.902898,-77.017474,Washington,20001,DC
348,58%,51%,480,2,Entire home/apt,0.0,1.0,1.0,159.0,$150.00,NaN,3,365,9,38.903948,-77.053441,Washington,20037,DC


In [3]:
# Mostrando informacion general de los datos. 
# Como: columnas, cuantos valores no nulos contiene cada columna y el tipo de dato
print(dc_listings.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3723 entries, 3390 to 3693
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   host_response_rate    3289 non-null   object 
 1   host_acceptance_rate  3109 non-null   object 
 2   host_listings_count   3723 non-null   int64  
 3   accommodates          3723 non-null   int64  
 4   room_type             3723 non-null   object 
 5   bedrooms              3702 non-null   float64
 6   bathrooms             3696 non-null   float64
 7   beds                  3712 non-null   float64
 8   price                 3723 non-null   float64
 9   cleaning_fee          2335 non-null   object 
 10  security_deposit      1426 non-null   object 
 11  minimum_nights        3723 non-null   int64  
 12  maximum_nights        3723 non-null   int64  
 13  number_of_reviews     3723 non-null   int64  
 14  latitude              3723 non-null   float64
 15  longitude             3

Como podemos observar: 
- Hay columnas que contienen valores _**NO númericos**_:
    - _room_type_
    - _city_
    - _state_
- Y columnas con valores númericos pero _**NO ordinales**_:
    - _latitude_
    - _longitude_
    - _zipcode_


A su vez tambien evitaremos cualquier columna que no describa directamente el espacio vital:
- _host_response_rate_
- _host_acceptance_rate_
- _host_listings_count_


In [4]:
# Eliminando columnas no utiles.
drop_columns = [ 'room_type', 'city', 'state', 'latitude', 'longitude', 'zipcode', 'host_response_rate', 'host_acceptance_rate', 'host_listings_count' ]
dc_listings = dc_listings.drop(drop_columns, axis=1)
dc_listings.isnull().sum()

accommodates            0
bedrooms               21
bathrooms              27
beds                   11
price                   0
cleaning_fee         1388
security_deposit     2297
minimum_nights          0
maximum_nights          0
number_of_reviews       0
dtype: int64

De las columnas restantes, hay algunas que tienen pocos valores nulos:
- _bedrooms_
- _bathrooms_
- _beds_

De las cuales podemos eliminar esas filas sin perder mucha informacion.

Pero tambien hay 2 columnas con un gran numero de valores perdidos:
- _cleaning_fee_
- _security_deposit_

En este caso no podemos eliminar simplemente las filas, ya que perderiamos mucha informacion. Por ello, vamos a eliminar <br>
por completo esas dos columas.

In [5]:
# Eliminando columnas con un gran numero de valores perdidos
dc_listings = dc_listings.drop([ 'cleaning_fee', 'security_deposit' ], axis=1)

# Eliminando filas con valores nulos
dc_listings = dc_listings.dropna(axis=0)

dc_listings.isnull().sum()

accommodates         0
bedrooms             0
bathrooms            0
beds                 0
price                0
minimum_nights       0
maximum_nights       0
number_of_reviews    0
dtype: int64

#### Normalizacion de Columnas

In [6]:
dc_listings.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
3390,1,1.0,1.0,1.0,99.0,2,1125,26
3168,2,1.0,1.0,1.0,65.0,1,5,4
3631,3,1.0,1.0,2.0,175.0,3,14,1
187,2,1.0,1.0,1.0,239.0,5,730,0
348,2,0.0,1.0,1.0,159.0,3,365,9


Como se puede observar, hay columnas con valores con mucha varianza, valores desmesurados. Y debido a como se calcula la _distancia euclideana_ <br>
podria ocasionar errores de distancia. 
Para evitar que una sola columna tenga demasiado impacto en la distancia, podemos _**normalizar**_ todas las columnas para que tengan una media de 0 <br>
y una desviacion estandar de 1.

_La **normalización** de los valores de cada columna preserva la distribucion de los valores de cada columana a su vez que alinea las escalas._

Esta es la formúla matemática que describe la transformacion que debe aplicarse a todos los valores de la columna: $$ Z_{i} = \frac{X_{i} - \mu}{\sigma} $$

Donde:
- $X_{i}$  es in valor de una columna especifica.
- $\mu$  es la media de todos los valores de la columna.
- $\sigma$  es a desviación estandar de todos los valores de la columna.

In [7]:
# Probando la normalizacion para entender la teoria.
first_transform = dc_listings['maximum_nights'] - dc_listings['maximum_nights'].mean()
normalized_column = first_transform / first_transform.std()
print(normalized_column.head(3))

# O de otra manera
normalized_column = first_transform / dc_listings['maximum_nights'].std()
print(normalized_column.head(3))

3390   -0.016573
3168   -0.016604
3631   -0.016604
Name: maximum_nights, dtype: float64
3390   -0.016573
3168   -0.016604
3631   -0.016604
Name: maximum_nights, dtype: float64


##### Aplicando esta trandormacion a todas las columnas de un DataFrame
Para ello se puede usar los metodos **mean()** y **std()** del DataFrame. Al utilizar estos métodos, se utilizan las medias y las desviaciones estándar de las columnas apropiadas para cada valor del DataFrame.

In [8]:
normalized_listings = ( dc_listings - dc_listings.mean() ) / (dc_listings.std())
normalized_listings['price'] =  dc_listings['price']
normalized_listings.head(3)

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
3390,-1.095499,-0.249467,-0.439151,-0.546858,99.0,-0.065038,-0.016573,0.372589
3168,-0.596544,-0.249467,-0.439151,-0.546858,65.0,-0.341375,-0.016604,-0.379894
3631,-0.097589,-0.249467,-0.439151,0.297345,175.0,0.211298,-0.016604,-0.482505


### Calcúlo de la Distancia Euclideana para el _**Caso Multivariante**_

Anteriormente utilizamos dos modelos univariantes de KNN, utilizando el atributo _**accomodates**_ para el primero y _**bathrooms**_ para el segundo. <br>
Ahora entrenaremos un modelo con ambos atributos. Recordando que la ecuacion de la _distancia euclideana_ esta definida como: $$ d = \sqrt{(q_{1}-p_{1})^{2} + (q_{2}-p_{2})^{2} + ... + (q_{n}-p_{n})^{2}} $$
Y bien para hacer el calculo en Python, ahora nos apoyaremos del modulo _**scipy**_, utilizando la funcion _**distance.euclidean()**_:

In [11]:
from scipy.spatial import distance

first_listing = [ -0.596544, -0.439151 ]
second_listing = [ -0.596544, -0.412923 ]

dist = distance.euclidean( first_listing, second_listing )

La funcion euclidean(), espera:
- Dos vectores, los cuales se deben representar con un objeto tipo _lista_ (lista de Python, array de NumPy o serie de Pandas).
- Ambos deben ser unidimensionales y tener el mismo numero de elementos.

In [14]:
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][ ['accommodates', 'bathrooms'] ]
fifth_listing = normalized_listings.iloc[4][ ['accommodates', 'bathrooms'] ]

first_fifth_distance = distance.euclidean( first_listing, fifth_listing )
print(f'La distancia entre el primer elemento y el quinto de nuestro conjunto de datos es: {first_fifth_distance}')

La distancia entre el primer elemento y el quinto de nuestro conjunto de datos es: 0.49895477658834086
